In [1]:
import pandas as pd
from functools import reduce
import datetime as dt
import re
import numpy as np

# Bringing in needed data

In [2]:
fhv = pd.read_csv('../data/FHV_Base_Aggregate_Report.csv') #NYC FHV Data

covidtests = pd.read_csv(
    '../github/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv') #Covid testing data

coviddeaths = pd.read_csv(
    '../github/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv') #Covid death data

vaccinedoses = pd.read_csv(
    '../COVID-19/data_tables/vaccine_data/us_data/time_series/vaccine_data_us_timeline.csv') #Covid vaccine data

# Putting together a dataframe with everything I need for the FHV VS Covid Dashboard

### fhv dataframe

In [3]:
#Datetime shenanighans
fhv['day'] = 1
fhv['Date'] = pd.to_datetime(fhv[['Year', 'Month', 'day']])
fhv['Date'] = fhv['Date'].apply(lambda x: x.strftime('%Y-%m')) #Makes it so the 'day' is removed

#Making a dataframe with the aggregates I need from fhv
fhv = fhv.groupby('Date')[['Total Dispatched Trips', 'Total Dispatched Shared Trips', 'Unique Dispatched Vehicles']].sum().reset_index()

#Final fhv dataframe
fhv

,Date,Total Dispatched Trips,Total Dispatched Shared Trips,Unique Dispatched Vehicles
0,2015-01,2699237,0,20345
1,2015-02,3023057,0,21171
2,2015-03,3182179,0,22137
3,2015-04,3769043,0,30986
4,2015-05,3284309,0,28296
...,...,...,...,...
77,2021-06,16264486,3743,103857
78,2021-07,16261550,19711,104332
79,2021-08,15695408,22432,105653
80,2021-09,15975710,25022,115028


### covidtests dataframe

In [4]:
#I have to make this dataframe workable
covidtests = covidtests.loc[(covidtests['Province_State'] == 'New York') 
                & (covidtests['Admin2'].isin(['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']))].reset_index(drop = True)

covidtests = covidtests.iloc[:, 12:] #Only keeps date columns
testdates = covidtests.columns.values.tolist() #Pulling the dates, which happen to be the column names

teststranspose = covidtests.transpose() 
testtotals = np.sum(teststranspose, axis = 1).tolist() #Adds up all values across row and puts it in a list

covidtests = pd.DataFrame(list(zip(testdates, testtotals)), columns = ['Date', 'Confirmed Cases'])

#Making the numbers per month instead of per day
covidtests['Date'] = pd.to_datetime(covidtests['Date'])
covidtests = covidtests.set_index('Date').groupby(pd.Grouper(freq = 'M')).max().reset_index() #This makes it so I take the final day of month's numbers only

#This finds the difference between rows since all numbers are originally a grand total
covidtests['Confirmed Cases'] = covidtests['Confirmed Cases'].diff()
covidtests = covidtests.drop([0, 1]).reset_index(drop = True) #First two rows are NaN and 0, so removes them

#Streamlining columns
covidtests['Date'] = covidtests['Date'].apply(lambda x: x.strftime('%Y-%m'))

#Final Covid tests dataframe
covidtests

,Date,Confirmed Cases
0,2020-03,43139.0
1,2020-04,128276.0
2,2020-05,34800.0
3,2020-06,11419.0
4,2020-07,10644.0
5,2020-08,8357.0
6,2020-09,10294.0
7,2020-10,20526.0
8,2020-11,52894.0
9,2020-12,110248.0


### coviddeaths dataframe

In [5]:
#I have to make this dataframe workable
coviddeaths = coviddeaths.loc[(coviddeaths['Province_State'] == 'New York') 
                & (coviddeaths['Admin2'].isin(['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']))].reset_index(drop = True)

coviddeaths = coviddeaths.iloc[:, 12:] #Only keeps date columns
deathdates = coviddeaths.columns.values.tolist() #Pulling the dates, which happen to be the column names

deathstranspose = coviddeaths.transpose() 
deathtotals = np.sum(deathstranspose ,axis = 1).tolist() #Adds up all values across row and puts it in a list

coviddeaths = pd.DataFrame(list(zip(deathdates, deathtotals)), columns = ['Date', 'Confirmed Deaths'])

#Making the numbers per month instead of per day
coviddeaths['Date'] = pd.to_datetime(coviddeaths['Date'])
coviddeaths = coviddeaths.set_index('Date').groupby(pd.Grouper(freq = 'M')).max().reset_index() #This makes it so I take the final day of month's numbers only

#This finds the difference between rows since all numbers are originally a grand total
coviddeaths['Confirmed Deaths'] = coviddeaths['Confirmed Deaths'].diff()
coviddeaths = coviddeaths.drop([0, 1]).reset_index(drop = True) #First two rows are NaN and 0, so removes them

#Streamlining columns
coviddeaths['Date'] = coviddeaths['Date'].apply(lambda x: x.strftime('%Y-%m'))

#Final Covid deaths dataframe
coviddeaths

,Date,Confirmed Deaths
0,2020-03,2192.0
1,2020-04,12732.0
2,2020-05,7354.0
3,2020-06,854.0
4,2020-07,351.0
5,2020-08,191.0
6,2020-09,125.0
7,2020-10,184.0
8,2020-11,263.0
9,2020-12,796.0


### vaccinedoses dataframe

In [6]:
#I need New York state only and all vaccine types 
vaccinedoses = vaccinedoses.loc[(vaccinedoses['Province_State'] == 'New York') & (vaccinedoses['Vaccine_Type'] == 'All')].reset_index(drop = True)

#Making the numbers per month instead of per day
vaccinedoses['Date'] = pd.to_datetime(vaccinedoses['Date'])
vaccinedoses = vaccinedoses.set_index('Date').groupby(pd.Grouper(freq = 'M')).max().reset_index() #This makes it so I take the final day of month's numbers only

vaccinedoses = vaccinedoses[['Date', 'Doses_admin']] #Makes it so I only keep my desired columns

#This finds the difference between rows since all numbers are originally a grand total
vaccinedoses.loc[-1] = ['2020-11-30', 0.0] #Adding a row so I can use .diff()
vaccinedoses.index = vaccinedoses.index + 1 #Shifting index
vaccinedoses = vaccinedoses.sort_index() #Sorting by index

vaccinedoses['Doses_admin'] = vaccinedoses['Doses_admin'].diff()
vaccinedoses = vaccinedoses.drop(0).reset_index(drop = True) #Removes inserted row


#Streamlining columns
vaccinedoses.columns = ['Date', 'Vaccine Doses Administered']
vaccinedoses['Date'] = vaccinedoses['Date'].apply(lambda x: x.strftime('%Y-%m'))

#Final Vaccine doses dataframe
vaccinedoses

,Date,Vaccine Doses Administered
0,2020-12,200000.0
1,2021-01,1655855.0
2,2021-02,2471613.0
3,2021-03,4901630.0
4,2021-04,6407511.0
5,2021-05,3968931.0
6,2021-06,2184957.0
7,2021-07,1159753.0
8,2021-08,1419956.0
9,2021-09,1554268.0


# Combining all my dataframes into one

In [7]:
alldataframes = [fhv, covidtests, coviddeaths, vaccinedoses] #Putting all dataframes I want to merge into a list

#Making the needed dataframe
dashboard = reduce(lambda left, right: pd.merge(left, right, on = ['Date'], how = 'outer'), alldataframes).fillna(0.0)

#Need to turn the dtype on all columns but 'Date' to integer so it looks nicer in the dashboard
cols = [i for i in dashboard.columns if i not in ['Date']]
for col in cols:
    dashboard[col] = dashboard[col].astype('int64')
    
#FHV Data ends in October, so I'll remove the bottom rows for consistency.
dashboard = dashboard[:-2]
    
dashboard.to_csv('../data/fhvdashdata.csv')

#### dashboard dataframe

In [8]:
dashboard

,Date,Total Dispatched Trips,Total Dispatched Shared Trips,Unique Dispatched Vehicles,Confirmed Cases,Confirmed Deaths,Vaccine Doses Administered
0,2015-01,2699237,0,20345,0,0,0
1,2015-02,3023057,0,21171,0,0,0
2,2015-03,3182179,0,22137,0,0,0
3,2015-04,3769043,0,30986,0,0,0
4,2015-05,3284309,0,28296,0,0,0
...,...,...,...,...,...,...,...
77,2021-06,16264486,3743,103857,6336,207,2184957
78,2021-07,16261550,19711,104332,20264,124,1159753
79,2021-08,15695408,22432,105653,57565,277,1419956
80,2021-09,15975710,25022,115028,50497,433,1554268
